In [ ]:
import numpy as np
import pandas as pd
import nltk
import re


from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn import preprocessing


In [3]:
import pandas as pd
raw_dataframe = pd.read_csv("nlpdata.txt",sep=",,,",header=None ,names=['inquiry','ans']) #Our data is separate by ,,,, so that's gonna be our sep, we don't need header, and inquiry and ans will be our column names

C:\Users\Archana k\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
raw_dataframe['ans']=raw_dataframe['ans'].str.strip() # Trying to clean all whitespaces, and blank char

In [4]:
raw_dataframe['inquiry'] = raw_dataframe['inquiry'].apply(lambda x: x.lower())#Converting all characters to lower, as capital char isn't gonna affect it's meaning
raw_dataframe['inquiry'] = raw_dataframe['inquiry'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x))) #Trying to replace all the unwanted characters, with space.

In [5]:
percentage_validation=0.20 #Keeping 20% of the data for validation set

## Main Recurrent Neural Network Architecture

In [6]:
max_words_capacity = 20000 #Total number of words, in the embedding
max_sentence_length = 30 # Max number of characters in the sentences

In [7]:
data=raw_dataframe.copy()

In [8]:
print(data['ans'].value_counts()) # Just checking if data isn't skewed.

tokenizer = Tokenizer(num_words=max_words_capacity, split=' ') # Splitting the sentences based on gaps 
tokenizer.fit_on_texts(data['inquiry'].values) # Fitting the tokenizer on text
X = tokenizer.texts_to_sequences(data['inquiry'].values) # Finally converting strings to integer
X = pad_sequences(X, maxlen=max_sentence_length) #Padding sequence to maximum length

what           609
who            402
unknown        272
affirmation    104
when            96
Name: ans, dtype: int64


In [12]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Y = data['ans']
convertor = preprocessing.LabelEncoder()
convertor.fit(Y)
Y=convertor.transform(Y) 
labels = to_categorical(np.asarray(Y))
print('Shape of data tensor:', X.shape)
print('Shape of label tensor:', labels.shape)


# split the data into a training set and a validation set
indices = np.arange(X.shape[0])
# np.random.seed(0)
np.random.shuffle(indices)
X = X[indices]
labels = labels[indices]
total_val_samples = int(percentage_validation * X.shape[0])

x_train = X[:-total_val_samples]
y_train = labels[:-total_val_samples]
x_val = X[-total_val_samples:]
y_val = labels[-total_val_samples:]

Found 3685 unique tokens.
Shape of data tensor: (1483, 30)
Shape of label tensor: (1483, 5)


In [13]:
embeddings_index = {}
f = open('glove.6B.100d.txt', encoding="utf8") #Download the glove embedding from here https://drive.google.com/open?id=1zwdYCA9dncuDE6dfYV7Tkph-5vRlMNfA+
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [14]:
word_dim=100

In [15]:
embedding_matrix = np.zeros((len(word_index) + 1, word_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [16]:

embedding_layer = Embedding(len(word_index) + 1,
                            word_dim,
                            weights=[embedding_matrix],
                            input_length=max_sentence_length,
                            trainable=False)

### Deleted Module

In [5]:
model.fit(x_train, y_train,
          batch_size=50,
          epochs=25,
          validation_data=(x_val, y_val))

NameError: name 'model' is not defined

In [2]:
model.load_weights('archana.h5')

NameError: name 'model' is not defined

In [7]:
test_sentence = tokenizer.texts_to_sequences(["What time does the train leave"])
test_sentence = pad_sequences(test_sentence, maxlen=max_sentence_length)
convertor.inverse_transform(np.argmax(model.predict(test_sentence)))

NameError: name 'tokenizer' is not defined